# Assignment 03

**Specifications and requirements** for each assignment include compliance with the [Programmer's Pact](../housekeeping/ProgrammerPact_Python_2026.pdf). In addition:
* you may **not** use the `in` operator for lists.
* you may **not** import any modules (ie no `import` statement allowed).

This assignment has #### problems:
* ...

## What to submit

A single Python file called `week03.py` with the code for the following problems.
